In [34]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np

class MT5:

    def get_ticks(symbol, number_of_data = 10000):
        from_date = datetime.now()
        ticks = mt5.copy_ticks_from(symbol, from_date, number_of_data, mt5.COPY_TICKS_ALL)
        df_ticks = pd.DataFrame(ticks)
        df_ticks["time"] = pd.to_datetime(df_ticks["time"], unit="s")
        df_ticks = df_ticks.set_index("time")
        return df_ticks

    def get_rates(symbol, number_of_data = 10000, timeframe=mt5.TIMEFRAME_D1):

        from_date = datetime.now()
        rates = mt5.copy_rates_from(symbol, timeframe, from_date, number_of_data)
        rates_frame = pd.DataFrame(rates)

        rates_frame['time'] = pd.to_datetime(rates_frame['time'], unit='s')
        rates_frame['time'] = pd.to_datetime(rates_frame['time'], format='%Y-%m-%d')
        rates_frame = rates_frame.set_index('time')
        
        return rates_frame

    def risk_reward_threshold(symbol, buy=True, risk=0.01, reward=0.02):
        #
        leverage = mt5.account_info().leverage

        # Extract the middle price between ask, and bid.
        # price = (mt5.symbol_info(symbol).ask + mt5.symbol_info(symbol).bid) / 2
        priceDecimal = mt5.symbol_info(symbol).ask

        # Counts the number of decimals, after the dot.
        nb_decimal = str(priceDecimal)[::-1].find(".")

        # Compute the variations in percentage
        var_down = risk/leverage
        var_up = reward/leverage

        # Computer the variations in absolute price
        # price_var_down = var_down*price
        # price_var_up = var_up*price

        if buy:
            price = mt5.symbol_info(symbol).ask
            price_var_down = var_down*price
            price_var_up = var_up*price

            tp = np.round(price + price_var_up, nb_decimal)
            sl = np.round(price - price_var_down, nb_decimal)
            print(f"PRICE: {price} \t Take Profit: {tp} \t Stop Loss: {sl}")
            return tp, sl
        else:
            price = mt5.symbol_info(symbol).bid
            price_var_down = var_down*price
            price_var_up = var_up*price

            tp = np.round(price - price_var_up, nb_decimal)
            sl = np.round(price + price_var_down, nb_decimal)
            print(f"PRICE: {price} \t Take Profit: {tp} \t Stop Loss: {sl}")
            return tp, sl

        


In [28]:
mt5.initialize()

True

In [26]:
something = MT5.get_ticks("EURUSD.a")
print(something)

Empty DataFrame
Columns: [bid, ask, last, volume, time_msc, flags, volume_real]
Index: []


In [29]:
something2 = MT5.get_rates("EURUSD.a")
print(something2)

               open     high      low    close  tick_volume  spread  \
time                                                                  
1987-03-06  1.06310  1.06370  1.06310  1.06340           91      50   
1987-03-09  1.05510  1.05650  1.05480  1.05500          331      50   
1987-03-10  1.05340  1.05380  1.05340  1.05340           81      50   
1987-03-11  1.04710  1.05110  1.04510  1.04730         1181      50   
1987-03-12  1.05100  1.05100  1.05100  1.05100            1      50   
...             ...      ...      ...      ...          ...     ...   
2023-04-03  1.08400  1.09167  1.07883  1.09024        74960       0   
2023-04-04  1.08878  1.09735  1.08829  1.09557        72209       0   
2023-04-05  1.09490  1.09696  1.08911  1.09070        72149       0   
2023-04-06  1.09006  1.09377  1.08846  1.09208        56851       0   
2023-04-07  1.09184  1.09241  1.08764  1.08972        24691       0   

            real_volume  
time                     
1987-03-06            0 

In [35]:
something3 = MT5.risk_reward_threshold("EURUSD.a", reward=0.01)
print(something3)

PRICE: 1.0903100000000001 	 Take Profit: 1.0910368733333335 	 Stop Loss: 1.0899465633333334
(1.0910368733333335, 1.0899465633333334)


In [36]:
something4 = MT5.risk_reward_threshold("EURUSD.a", reward=0.03)
print(something4)

PRICE: 1.0903100000000001 	 Take Profit: 1.09140031 	 Stop Loss: 1.0899465633333334
(1.09140031, 1.0899465633333334)
